## Prepare Data for Statistical Analyses in R using Pandas

In [1]:
import numpy as np
import scipy.io as sio
import pandas as pd

In [2]:
subjects = ['AB156','AB185','AB186','AB188','AB189','AB190','AB191','AB192','AB193','AB194']
classifiers = ['LDA','SVM','ANN','SRC']
df = pd.DataFrame()

In [3]:
for delaystr in ['0Delay']:      
    erates_temp = sio.loadmat('AllSubs_' + delaystr + '_SensorClassifierCombos.mat')
    erates_all = erates_temp['withinerror']
    erates_ss = erates_temp['withinsserror']
    erates_t = erates_temp['withinterror']    
    sensors = erates_all[0,1:]
    sensornums = np.asarray(range(1,34))
    for subind in range(0,10):
        for classifierind in np.arange(0,4):
            subjecttemp = np.tile(subjects[subind],[33,1])
            classifiertemp = np.tile(classifiers[classifierind],[33,1])
            if delaystr == '0Delay':
                delaytemp = np.tile(0,[33,1])
            sensortemp = sensors
            sensornumtemp = sensornums
            e_alltemp = [None]*33
            e_sstemp = [None]*33
            e_ttemp = [None]*33
            lateralitytemp = [None]*33
            emgtemp = [0]*33
            goniotemp = [0]*33
            imutemp = [0]*33
            modalitytemp = [None]*33
            for sensorind in np.arange(0,33):  
                e_alltemp[sensorind] = 100.*erates_all[classifierind+1,sensorind+1].ravel()[subind]
                e_sstemp[sensorind] = 100.*erates_ss[classifierind+1,sensorind+1].ravel()[subind]
                e_ttemp[sensorind] = 100.*erates_t[classifierind+1,sensorind+1].ravel()[subind]
                if sensorind in [0,1,2,3,4,5,6,17,18,21,22,25,26]:
                    lateralitytemp[sensorind] = 1 # Ipsilateral
                elif sensorind in [29,30,31,32]:
                    lateralitytemp[sensorind] = -1 # Contralateral
                else:
                    lateralitytemp[sensorind] = 2 # Bilateral
                if sensorind in [0,13,29]:
                    modalitytemp[sensorind] = 1 # EMG
                if sensorind in [1,14,30]:
                    modalitytemp[sensorind] = 2 # GONIO
                if sensorind in [2,15,31]:
                    modalitytemp[sensorind] = 3 # IMU
                if sensorind in [6,16,32]:
                    modalitytemp[sensorind] = 4 # ALL
                if sensorind in [0,3,4,6,7,8,9,10,11,12,13,16,17,18,19,20,29]:
                    emgtemp[sensorind] = 1 # Contains some EMG
                if sensorind in [1,3,5,6,7,8,9,10,11,12,14,16,21,22,23,24,30]:
                    goniotemp[sensorind] = 1 # Contains some GONIO
                if sensorind in [2,4,5,6,7,8,9,10,11,12,15,16,25,26,27,28,31]:
                    imutemp[sensorind] = 1 # Contains some IMU

            tempdict = {'Subject': subjecttemp.ravel(),'Classifier': classifiertemp.ravel(),'Delay': delaytemp.ravel(),'Sensors': sensortemp.ravel(),'SensorKey': sensornumtemp.ravel(),'Modality': modalitytemp,'EMG': emgtemp,'GONIO': goniotemp,'IMU': imutemp,'Laterality': lateralitytemp,'Overall': np.asarray(e_alltemp),'SS': np.asarray(e_sstemp),'T': np.asarray(e_ttemp)}
            tempdf = pd.DataFrame(tempdict)
            df = df.append(tempdf,ignore_index=True)

In [4]:
cols = ['Subject','Classifier','Delay','Sensors','SensorKey','Modality','EMG','GONIO','IMU','Laterality','Overall','SS','T']
df = df[cols]

In [5]:
writer = pd.ExcelWriter('AllSubsErrors_ForStats.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

In [ ]:
GONIO_df = df[(df.Sensors == 'KNEE(I)') | (df.Sensors == 'ANKLE(I)') | (df.Sensors == 'GONIO(I)') | (df.Sensors == 'GONIO(I)/KNEE(C)') | (df.Sensors == 'GONIO(I)/ANKLE(C)') | (df.Sensors == 'BILAT GONIO')]
EMG_df = df[(df.Sensors == 'EMG LL(I)') | (df.Sensors == 'EMG UL(I)') | (df.Sensors == 'EMG(I)') | (df.Sensors == 'EMG(I)/EMG LL(C)') | (df.Sensors == 'EMG(I)/EMG UL(C)') | (df.Sensors == 'BILAT EMG')]
IMU_df = df[(df.Sensors == 'SHANK(I)') | (df.Sensors == 'THIGH(I)') | (df.Sensors == 'IMU(I)') | (df.Sensors == 'IMU(I)/SHANK(C)') | (df.Sensors == 'IMU(I)/THIGH(C)') | (df.Sensors == 'BILAT IMU')]
subsetdf = df[(df.Sensors == 'EMG(I)') | (df.Sensors == 'GONIO(I)') | (df.Sensors == 'IMU(I)') | (df.Sensors == 'IPSI ALL') | (df.Sensors == 'BILAT EMG') | (df.Sensors == 'BILAT GONIO') | (df.Sensors == 'BILAT IMU') | (df.Sensors == 'BILAT ALL')]

##### SFS to Excel

In [9]:
subjects = ['AB156','AB185','AB186','AB188','AB189','AB190','AB191','AB192','AB193','AB194']
df1 = pd.DataFrame()

In [10]:
sfs_erates_temp = sio.loadmat('contra_kin_sfs_results.mat')

contra_erates_all = sfs_erates_temp['contra_err']
contra_erates_ss = sfs_erates_temp['contra_ss_err']
contra_erates_t = sfs_erates_temp['contra_t_err']    

In [11]:
for subind in range(0,10):
    subjecttemp = np.tile(subjects[subind],[5,1])
    sfs_e_alltemp = [None]*5
    sfs_e_sstemp = [None]*5
    sfs_e_ttemp = [None]*5
    iter_temp = [0]*5
    for iteration in np.arange(0,5):
        sfs_e_alltemp[iteration] = contra_erates_all[iteration,subind]
        sfs_e_sstemp[iteration] = contra_erates_ss[iteration,subind]
        sfs_e_ttemp[iteration] = contra_erates_t[iteration,subind]
        iter_temp[iteration] = iteration
        
    tempdict1 = {'Subject': subjecttemp.ravel(),'Iteration':np.asarray(iter_temp),'Overall': np.asarray(sfs_e_alltemp),'SS': np.asarray(sfs_e_sstemp),'T': np.asarray(sfs_e_ttemp)}
    tempdf1 = pd.DataFrame(tempdict1)
    df1 = df1.append(tempdf1,ignore_index=True)

cols = ['Subject','Iteration','Overall','SS','T']
df1 = df1[cols]

In [12]:
writer1 = pd.ExcelWriter('AllSubsSFS_ForStats.xlsx')
df1.to_excel(writer1,'Sheet1')
writer1.save()